In [ ]:
from rdflib import Namespace

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes

In [ ]:
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://")

# create the model w/ a namespace
BLDG = Namespace("urn:ex/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

things = []

In [ ]:
# load libraries
# NREL templates for 223P
nrel_lib = Library.load(directory="../libraries/ashrae/223p/nrel-templates")
# 223P ontology
s223 = Library.load(ontology_graph="../libraries/ashrae/223p/ontology/223p.ttl")

In [ ]:
# get some templates from the library
mau_templ = nrel_lib.get_template_by_name("makeup-air-unit").inline_dependencies()
vav_templ = nrel_lib.get_template_by_name("vav-reheat").inline_dependencies()
duct_templ = nrel_lib.get_template_by_name("duct").inline_dependencies()
zone_templ = nrel_lib.get_template_by_name("hvac-zone").inline_dependencies()
space_templ = nrel_lib.get_template_by_name("hvac-space").inline_dependencies()
fcu_templ = nrel_lib.get_template_by_name("fcu").inline_dependencies()
chws_templ = nrel_lib.get_template_by_name("chilled-water-system").inline_dependencies()
hws_templ = nrel_lib.get_template_by_name("hot-water-system").inline_dependencies()

In [ ]:
# create a makeup air unit
mau = mau_templ.evaluate({"name": BLDG.MAU, "air-supply": BLDG.MAU_Supply})
things.append(mau)

In [ ]:
# make 2 VAVs, connect them to the MAU via ducts
vav1 = vav_templ.evaluate({"name": BLDG["VAV-1"], "air-in": BLDG["VAV-1-in"]})
mau_to_vav1 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-1-in"]})
things.append(vav1)
things.append(mau_to_vav1)

vav2 = vav_templ.evaluate({"name": BLDG["VAV-2"], "air-in": BLDG["VAV-2-in"]})
things.append(vav2)
mau_to_vav2 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-2-in"]})
things.append(mau_to_vav2)

In [ ]:
# VAV1 goes to Zone 1
zone1 = zone_templ.evaluate({"name": BLDG["zone1"]})
things.append(zone1)
zone1space1 = space_templ.evaluate({"name": BLDG["zone1space1"], "zone": BLDG["zone1"]})
things.append(zone1space1)
vav1_to_zone1 = duct_templ.evaluate({"a": BLDG["VAV-1-out"], "b": BLDG["zone1-in"]})
things.append(vav1_to_zone1)

In [ ]:
# VAV2 goes to FCU1
fcu1 = fcu_templ.evaluate({"name": BLDG["fcu1"]})
vav2_to_fcu1 = duct_templ.evaluate({"a": BLDG["VAV-2-out"], "b": BLDG["fcu1-in"]})
things.extend([fcu1, vav2_to_fcu1])

In [ ]:
# FCU1 goes to Zone 2
zone2 = zone_templ.evaluate({"name": BLDG["zone2"]})
zone2space1 = space_templ.evaluate({"name": BLDG["zone2space1"], "zone": BLDG["zone2"]})
fcu1_to_zone2 = duct_templ.evaluate({"a": BLDG["fcu1-out"], "b": BLDG["zone2-in"]})
things.extend([zone2, zone2space1, fcu1_to_zone2])

In [ ]:
# add chws, hws
chws = chws_templ.evaluate({"name": BLDG["CHWS"]})
hws = hws_templ.evaluate({"name": BLDG["HWS"]})
things.extend([chws, hws])

In [ ]:
# fill in all the extra parameters with invented names
for templ in things:
    print(templ.name)
    _, graph = templ.fill(BLDG, include_optional = True)
    bldg.add_graph(graph)

In [ ]:
# look at finished model
print(bldg.graph.serialize())
bldg.graph.serialize("output.ttl")

In [ ]:
# validate against 223P
ctx = bldg.validate([s223.get_shape_collection()])
print(ctx.valid)
print(ctx.report_string)

In [ ]:
bldg.add_graph(mau_templ.fill(BLDG)[1])

In [ ]:
ctx = bldg.validate([s223.get_shape_collection()])
print(ctx.valid)
print(ctx.report_string)

In [ ]:
print(mau_templ.fill(BLDG)[1].serialize())
